Hi there! It's my first time to share notebook here. Though this notebook doesn't do well in this competition, I want to share it as a reference here(Maybe it may do some help for someone uh?) Upvote if you think it's useful :)✌

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tqdm

In [ ]:
customer_df = pd.read_csv('./data/customers.csv')
sub_df = pd.read_csv('./data/sample_submission.csv')
trans_df = pd.read_csv('./data/transactions_train.csv', usecols=['article_id', 'customer_id', 't_dat'])
article_df = pd.read_csv('./data/articles.csv', )
trans_df['t_dat'] = pd.to_datetime(trans_df['t_dat'])

In [ ]:
article_id_dict = dict(zip(article_df.article_id.values, article_df.index))
id_article_dict = dict(zip(article_df.index, article_df.article_id.values))

customer_id_dict = dict(zip(customer_df.customer_id.values, customer_df.index))
id_customer_dict = dict(zip(customer_df.index, customer_df.customer_id.values))

In [ ]:
trans_df1 = trans_df[['article_id', 'customer_id']].copy()
trans_df1['val'] = 1

# Negative sampling

Negative sample with popularity theory that popular items are recommended while user does not choose

In [ ]:
def get_general_item(df:pd.DataFrame, N:int):
    """
    To get popular items
    """
    last_ts = df.t_dat.max()
    df['day'] = df.t_dat.dt.dayofweek
    df['new_t_dat'] = df.t_dat - pd.TimedeltaIndex(df.day - 1, 'D')
    df.loc[df.day >= 2, 'new_t_dat'] = df.loc[df.day >= 2, 'new_t_dat'] + pd.TimedeltaIndex(np.ones(len(df.loc[df.day >= 2, 'new_t_dat'])) * 7, 'D')
    
    tmp = df.groupby(['article_id', 'new_t_dat'])['day'].count().reset_index().rename(columns={'day': 'cnt'})
    tmp1 = tmp[tmp.new_t_dat == last_ts][['article_id', 'cnt']]
    df = pd.merge(df, tmp, on=['article_id', 'new_t_dat'], how='left')
    df = pd.merge(df, tmp1, on=['article_id'], how='left', suffixes=('', '_target'))

    df['cnt_target'].fillna(0, inplace=True)
    df['quotient'] = df['cnt_target'] / df['cnt']
    
    general_pred = df.groupby(['article_id'])['quotient'].sum().nlargest(N).index.tolist()
    return general_pred

In [ ]:
general_items2000 = get_general_item(trans_df.copy(), 2000)
general_items200 = get_general_item(trans_df.copy(), 200)

In [ ]:
tmp = trans_df1.groupby(['customer_id']).agg({'article_id':list}).reset_index()

In [ ]:
# Negative sample with popularity theory that popular items are recommended while user does not choose
def negative_sample(df, N=5):
    add_items = []
    add_users = []
    temp = pd.DataFrame(columns=['article_id', 'customer_id'])
    for i in tqdm.tqdm(range(len(tmp))):
        j = 0
        items = df.loc[i, 'article_id']
        cur_user = df.loc[i, 'customer_id']
        for general_item in general_items200:
            if j == N:
                break
            if general_item not in items:
                add_items.append(general_item)
                add_users.append(cur_user)
                j += 1
        if j < N:
            for general_item in general_items2000:
                if j == N:
                    break
            if general_item not in items:
                add_items.append(general_item)
                add_users.append(cur_user)
                j += 1
    temp['article_id'] = add_items
    temp['customer_id'] = add_users
    temp['val'] = 0

    return temp

In [ ]:
trans_df1 = trans_df1.append(negative_sample(tmp, 5))

# Funk-SVD

In [ ]:
class FunkSVD(nn.Module):
    def __init__(self, user_len, item_len, emb_len):
        super().__init__()
        self.user_len = user_len
        self.item_len = item_len
        self.emb_len = emb_len
        self.item_embeddings = nn.Embedding(self.item_len+1, self.emb_len)
        self.user_embeddings = nn.Embedding(self.user_len, self.emb_len)
    
    def forward(self, x):
        user_emb = self.user_embeddings(x[:, 1])
        item_emb = self.item_embeddings(x[:, 0])
        return torch.sum(user_emb * item_emb, dim=1, keepdim=True)

# Begin training

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


trans_df1['article_id'] = trans_df1['article_id'].map(article_id_dict)
trans_df1['customer_id'] = trans_df1['customer_id'].map(customer_id_dict)


X = torch.tensor(trans_df1[['article_id', 'customer_id']].values, dtype=torch.long).to(device)
y = torch.tensor(trans_df1['val'].values, dtype=torch.long).to(device)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, 256, shuffle=True)

loss = nn.MSELoss()
model = FunkSVD(len(customer_df), len(article_df), 10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, loss, optimizer, dataloader, epoch):
    model.train()
    for i in range(epoch):
        for X, y in tqdm.tqdm(dataloader):
            y_hat = model(X)
            l = loss(y_hat.squeeze(1), y.float())

            model.zero_grad()
            l.backward()
            optimizer.step()

In [ ]:
train(model, loss, optimizer, dataloader, 10)

# Compute final results

In [ ]:
user_embeddings = model.user_embeddings.weight.to(torch.device('cpu')).detach().numpy()
item_embeddings = model.item_embeddings.weight.to(torch.device('cpu')).detach().numpy()

In [ ]:
for i in tqdm.trange(len(sub_df)):
    cur_customer = customer_id_dict[sub_df.loc[i, 'customer_id']]
    score = item_embeddings@(user_embeddings[cur_customer])
    res = ' '.join(['0' + str(id_article_dict[x]) for x in score.argsort()[::-1][:12].tolist()])
    sub_df.loc[i, 'prediction'] = res

In [ ]:
sub_df.to_csv('my_sub.csv', index=False)

reference notebook: [https://www.kaggle.com/code/hervind/h-m-faster-trending-products-weekly](https://www.kaggle.com/code/hervind/h-m-faster-trending-products-weekly)